### Goldsmiths University of London
### Author....: Carlos Manuel de Oliveira Alves
### Student...: cdeol003
### Created...: 08/02/2023
### FYP.......: NeuroCredit

In [ ]:
# Importing the pandas library to read the data
import pandas as pd

In [ ]:
# Reading the data with credit loans data
data = pd.read_csv('data.csv')

In [ ]:
# Printing the first 5 rows of the data
data.head().T

In [ ]:
# Describing the dataset with the mean, standard deviation, min, max, etc.
data.describe().T

In [ ]:
# Import Necessary Libraries with sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Split the data into train and test sets
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Print the number of rows and columns in the train and test sets
print('Train set: ', train.shape)
print('Test set: ', test.shape)

In [ ]:
# Create Logistic Regression model
logistic_regression = LogisticRegression()

# One-hot encode the string values in the train and test datasets
encoder = OneHotEncoder(handle_unknown='ignore')
X_train = encoder.fit_transform(train.drop(columns=['approval_status']))
X_test = encoder.transform(test.drop(columns=['approval_status']))

# Create y_train with the target column from the training set
y_train = train['approval_status']

# Initialize the LabelEncoder
encoder = LabelEncoder()

# Fit and transform the y_train labels using the LabelEncoder
y_train = encoder.fit_transform(y_train)

# Train the model on the training set
logistic_regression.fit(X_train, y_train)

# Make predictions for the test set
predictions = logistic_regression.predict(X_test)

# Create y_test with the target column from the test set
y_test = test['approval_status']

# Transform the y_test labels using the LabelEncoder (without re-fitting)
y_test = encoder.transform(y_test)

# Print the first 5 predictions
print("First 5 Predictions:")
for i, prediction in enumerate(predictions[:5]):
    print(f"{prediction} - The model predicts that the {i + 1} example belongs to class {prediction}.")

# Print the first 5 probabilities of the predictions
probabilities = logistic_regression.predict_proba(X_test)[:5]
print("\nFirst 5 Probabilities:")
for i, probability in enumerate(probabilities):
    class_0_prob = probability[0]
    class_1_prob = probability[1]
    class_0_prob_pct = round(class_0_prob * 100, 2)
    class_1_prob_pct = round(class_1_prob * 100, 2)
    print(f"[{class_0_prob:.2f}, {class_1_prob:.2f}] - {class_1_prob_pct:.2f}% probability of class 1, {class_0_prob_pct:.2f}% probability of class 0.")

# Evaluate the performance of the model
acc = accuracy_score(y_test, predictions)
prec = precision_score(y_test, predictions)
rec = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

# Print the performance metrics
print('\nAccuracy...: {:.2f} %'.format(acc*100))
print('Precision..: {:.2f} %'.format(prec*100))
print('Recall.....: {:.2f} %'.format(rec*100))
print('F1 Score...: {:.2f} %'.format(f1*100))

In [ ]:
# Plot the confusion matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# Create the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Plot the confusion matrix
plt.figure(figsize=(5, 5))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Approved', 'Denied'], rotation=45)
plt.yticks(tick_marks, ['Approved', 'Denied'])
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# Plot the ROC curve
from sklearn.metrics import roc_curve, auc

# Compute the false positive rate, true positive rate and thresholds
fpr, tpr, thresholds = roc_curve(y_test, predictions)

# Compute the area under the curve
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Plot the precision-recall curve
from sklearn.metrics import precision_recall_curve

# Compute the precision and recall
precision, recall, thresholds = precision_recall_curve(y_test, predictions)

# Plot the precision-recall curve
plt.figure(figsize=(5, 5))
plt.plot(recall, precision, color='darkorange', lw=2, label='Precision-Recall curve')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Plot the learning curve
from sklearn.model_selection import learning_curve

# Create the learning curve
train_sizes, train_scores, test_scores = learning_curve(logistic_regression, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1, train_sizes=np.linspace(0.01, 1.0, 50))

# Compute the mean and standard deviation of the training and test scores
train_mean = np.mean(train_scores, axis=1)

# Compute the mean and standard deviation of the training and test scores
train_std = np.std(train_scores, axis=1)

# Plot the learning curve
plt.figure(figsize=(5, 5))
plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training Accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.grid()
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.8, 1.0])
plt.show()

In [ ]:
# Plot the validation curve
from sklearn.model_selection import validation_curve

# Create the validation curve
param_range = np.logspace(-6, -1, 5)
train_scores, test_scores = validation_curve(logistic_regression, X_train, y_train, param_name='C', param_range=param_range, cv=5, scoring='accuracy', n_jobs=-1)

# Compute the mean and standard deviation of the training and test scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the validation curve
plt.figure(figsize=(5, 5))
plt.plot(param_range, train_mean, color='blue', marker='o', markersize=5, label='Training Accuracy')
plt.fill_between(param_range, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(param_range, test_mean, color='green', linestyle='--', marker='s', markersize=5, label='Validation Accuracy')
plt.fill_between(param_range, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.grid()
plt.xscale('log')
plt.legend(loc='lower right')
plt.xlabel('Parameter C')
plt.ylabel('Accuracy')
plt.ylim([0.8, 1.0])
plt.show()

In [ ]:
# Plot F1 score vs. threshold
from sklearn.metrics import f1_score

# Compute the F1 score for each threshold
f1_scores = []
thresholds = np.linspace(0, 1, 100)
for threshold in thresholds:
    y_pred = (logistic_regression.predict_proba(X_test)[:, 1] >= threshold).astype(bool)
    f1_scores.append(f1_score(y_test, y_pred))

# Plot the F1 score vs. threshold
plt.figure(figsize=(5, 5))
plt.plot(thresholds, f1_scores, color='darkorange', lw=2)
plt.grid()
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.show()

In [ ]:
# Plot precision vs. recall
from sklearn.metrics import precision_recall_curve

# Compute the precision and recall
precision, recall, thresholds = precision_recall_curve(y_test, predictions)

# Plot the precision vs. recall
plt.figure(figsize=(5, 5))
plt.plot(recall, precision, color='darkorange', lw=2)
plt.grid()
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [ ]:
# Create a dictionary with values for the input variables to have a prediction = Approved
input_dict = {'credit_history': 'Good',
            'employment_status': 'Employed',
            'collateral': 'House',
            'payment_history': 'Good',
            'type_of_credit_accounts': 'Other',
            'public_records_and_collections': 'Judgement',
            'purpose_of_loan': 'Other',
            'income': 44707,
            'assets_value': 35464,
            'debt_to_income_ratio': 74,
            'length_of_credit_history': 38,
            'number_of_credit_inquiries': 1,
            'number_of_credit_accounts': 4,
            'number_of_credit_accounts_opened_last_12_months': 4,
            'current_balance_of_credit_accounts': 14550,
            'total_credit_limit': 14170,
            'total_credit_utilization': 13,
            'loan_amount': 1464,
            'saving_account_balance': 7778}

# Convert the dictionary to a pandas DataFrame with a single row
input_df = pd.DataFrame([input_dict])

# Encode the input data using the LabelEncoder
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
input_df_encoded = encoder.fit_transform(input_df)

# Scale the input data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
input_df_scaled = scaler.fit_transform(input_df_encoded)

# Make predictions for the input data using the trained logistic regression model
predictions = logistic_regression.predict(input_df_scaled)

# Decode the predicted approval status using the LabelEncoder
predicted_approval_status = encoder.inverse_transform(predictions)

# Print the predicted approval status
print("Predicted Approval Status: {}".format(predicted_approval_status[0]))